## Densidad de materia oscura

### Informacion por utilizar

In [2]:
#importamos las herramientas necesarias
import pynbody
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
from scipy import optimize
from astropy import units as unit
import sys
from matplotlib.animation import FuncAnimation

In [3]:
#importamos los datos
data_nuevo = '/media/danna01/Disk/SNAPSHOTS/Galaxias_Nuevas'
data_viejo = '/media/danna01/Disk/SNAPSHOTS/Galaxias_Viejas'
#datadir = '/media/juan/Pesquisa/Simulations_Chris_2021'

In [4]:
#A continuacion se lista la informacion importante para este notebook

# Masa Total de materia oscura
# Densidad de la materia oscura

In [5]:
# Estos son los diccionarios que definen los limites que seran utilizados para los plots de las galaxias para los
# calculos de densidad de materia oscura y la masa acumulada a diferentes radios para la amteria oscura tambien
dicti = {}
galaxies = ['G0', 'G1', 'G2', 'G3']
# Estos son los limites para graficar la masa acumulada y la densidad; sus valores corresponden a 10 veces el valor
# del scale - lenght de cada galaxia.
limites_materia_oscura = {'G0':(0,11), 'G1':(0,15), 'G2':(0,19), 'G3':(0,28)} #kpc
# Aca observamos los limites que se usaran para calcular el valor de 'h' que nos indicara que tan bueno es el ajuste
# limites --> De que dependen?

### Funciones

#### Relacion masa acumulada vs. radio

In [6]:
# En esta celda estamos definiendo la masa acumulada de la materia oscura

def accumulated_mass(pos,mass,n,limits):
    # con esta linea calculamos el radio de la esfera en la que realizaremos el calculo
    r = np.sqrt((pos**2).sum(axis=1))
    # con esta linea se genera un histograma, cuyos rectangulos son equivalentes a los radios de los cascarones de
    # las esferas que consideremos para este calculo; r hace referencia a los radios, n al numero de divisiones,
    # weights le asigna un valor a cada punto, en este caso relacionado a la masa y range son los limites.
    histo   = np.histogram(r,n, weights= mass, range=limits)
    # con esta linea definimos que los radios que consideraremos seran los que se ubiquen en el extremo de cada
    # rectangulo del histograma
    bins     = histo[1][1:]
    # note que bins corresponde a los radios e histo[0] al valor de la masa acumulada hasta dicho radio
    return bins, histo[0]

#### Relacion densidad vs. radio

In [15]:
# En esta celda se va a definir la funcion de densidad
def density_dm(x,y):
    # En esta linea debemos aclarar que lo que aca se considera x y y corresponde al radio y a la amsa acumulada
    # que se calculo en la funcion anterior
    x,y = accumulated_mass(data.dm['pos'],data.dm['mass'],n=int(limites_materia_oscura[gal][1]*4),limits=limites_materia_oscura[gal])
    x_cubico = x**3
    # con esta linea vamos a generar un arreglo igual al de los radios calculados pero desplazado una posicion
    # para poder sumar los dos radios que son continuos; ademas el primer valor es remplazado con un cero.
    x2 = np.roll(x_cubico,1)
    x2[0]=0
    # con esta linea sumamos los dos radios continuos el uno del otro y los dividimos en dos para hallar un promedio
    # de sus valores
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    bins_middle = (x + np.roll(x,1) )/2
    # con esta linea vamos a generar un arreglo igual a las masas acumuladas calculadas pero desplazadas una posicion
    # para poder restarlas y poder hallar el valor de la masa en medio de los dos radios que se consideren, es decir,
    # calcular la masa de cada cascaron. Nuevamente el primer valor se remplaza con cero.
    y2 = np.roll(y,1)
    y2[0]=0
    # con esta linea vamos a hacer la resta entre las masas para hallar la masa en el cascaron.
    shell_mass = y - y2
    # con esta linea calculamos la densidad dividiendo la masa entre el volumen.
    # \rho = (4/3)*pi*R^3
    dens = 3* shell_mass / ((4*np.pi)* (x_cubico - x2) ) # ESTO SI !!!
    
    dens = (3*shell_mass)/((4*np.pi)*(bins_middle)) # ESTO NO !!!!
    return bins_middle, shell_mass, dens

#### Funciones para las graficas

In [12]:
# Con esta celda se va a generar la funcion de los plots de masa acumulada.
def plot_mass(bins,mass,filename):
    # con esta linea genramos la figura, con el tamaño que se desee
    fig = plt.figure(figsize=(10, 7))
    # aca simplemente definimos las variables que vamos a graficar, en este caso son los radios contra la masa
    # acumulada de cada radio y el estilo requerido
    plt.plot(bins,mass,'-k')
    # aca definimos el titulo, los titulos de los ejes, la cuadricula y la forma en que vamos a guardar la 
    # grafica.
    plt.title('Masa acumulada de la Materia Oscura', fontsize= 18)
    plt.xlabel('Radio [Kpc]',fontsize=20)
    plt.ylabel('Masa Acumulada (Msol)')
    plt.grid()
    plt.savefig(filename)
    plt.clf()
    plt.close()

In [13]:
# Con esta linea vamos a realizar la grafica para la densidad.
def plot_density(bins,dens,filename):
    # Definimos la figura con el tamaño que se desee
    fig = plt.figure(figsize=(10, 7))
    # Con esta linea generamos el plot de los radios y el logaritmo de la densidad, en el eje y.
    plt.plot(bins, np.log10(dens), '-ok')
    # Aca definimos otras caracteristicas como el titulo, el titulo de los ejes, la cuadricula, y 
    # como se va a guardar la imagen
    plt.title('Perfil de densidad de materia oscura', fontsize = 18)
    plt.xlabel('Radio [Kpc]', fontsize = 20)
    plt.ylabel(r'$\rho$ [$1x10^{10}$ Msol/$kpc^{2}$]', fontsize = 20)
    plt.grid()
    plt.savefig(filename)
    plt.clf()
    plt.close()

#### Generacion de graficas

In [16]:
#En esta celda vamos a generar los plots deseados
# En primer lugar definimos los limites de los snapshots que vamos a considerar y el espacio entre los mismos.
minsnap = 100
maxsnap = 300
stepsnap = 100

# definimos la fuente de los datos, para esto usamos los diccionarios que definimos mas arriba
for gal in galaxies:
    for snap in np.arange(minsnap,maxsnap,stepsnap): 
        snap = str(snap).zfill(3)
        # Aca definimos la ruta en la cual estan almacenados los snapshots
        data = pynbody.load(data_nuevo+'/'+gal+'/snapshot_'+snap+'.hdf5')
        N_star = data.star['mass'].size
        
        # Aca vamos a generar cada uno de los plots, para lo cual primero debemos definir
        # cada una de las funciones de masa acumulada y densidad.
        #Funcion masa acumulada (de la cual obtenemos los valores de radio y masa acumulada)
        bins, mass = accumulated_mass(data.dm['pos'],data.dm['mass'],n=int(limites_materia_oscura[gal][1]*4),limits=limites_materia_oscura[gal])
        # Funcion densidad (de la cual obtenemos los radios promedio y la densidad)
        bins_middle, shell_mass, dens = density_dm(bins,mass)
        # Funcion para graficar la masa acumulada vs el radio
        plot_mass(bins,mass,filename='Materia_oscura/Materia_Oscura_Acumulada_'+gal+'_dm_'+snap+'.jpg')
        # Funcion para graficar la densidad de la materia oscura en escala logaritmica vs el radio promedio
        plot_density(bins_middle,dens,filename='Materia_oscura/Materia_Oscura_Densidad_'+gal+'_dm_'+snap+'.jpg')

/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No unit information found: using gadget-units.
  warnings.warn(
<ipython-input-13-0fc0d2458f42>:6: RuntimeWarning: invalid value encountered in log10
  plt.plot(bins, np.log10(dens), '-ok')
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:508: RuntimeWarning: No unit information found!
  warnings.warn("No unit information found!", RuntimeWarning)
/home/danna01/.local/lib/python3.8/site-packages/pynbody/snapshot/gadgethdf.py:572: RuntimeWarning: No unit information found: using gadget-units.
  warnings.warn(
<ipython-input-13-0fc0d2458f42>:6: RuntimeWarning: divide by zero encountered in log10
  plt.plot(bins, np.log10(dens), '-ok')
<ipython-input-13-0fc0d2458f42>:6: Runtim

#### Descubriendo np.roll

In [46]:
x = np.arange(10)
x2 = np.roll(x,1)
print(x)
print(x2)
resta = x - x2
print(resta)

[0 1 2 3 4 5 6 7 8 9]
[9 0 1 2 3 4 5 6 7 8]
[-9  1  1  1  1  1  1  1  1  1]
